In [1]:
import sys
sys.path.append('src/utils')
import mesh_tools as mt
from SDF_CNN import CNN_3d_multiple_split
from CNN_to_PoNQ_or_lite import CNN_to_PoNQ
from ABC_dataset import make_mask_close
import torch
from meshplot import plot
import igl

In [3]:
state_dict = 'data/pretrained_PoNQ_ABC.pt'
example_mesh = 'data/bunny.obj'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
model = CNN_3d_multiple_split(device=device)
model.load_state_dict(torch.load(state_dict, map_location=device))
model.to(device)
model.eval()

CNN_3d_multiple_split(
  (encoder): Sequential(
    (0): Conv3d(1, 128, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (3): LeakyReLU(negative_slope=0.01, inplace=True)
    (4): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (7): LeakyReLU(negative_slope=0.01, inplace=True)
    (8): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (9): LeakyReLU(negative_slope=0.01, inplace=True)
    (10): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (11): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (decoder_points): SDF_decoder(
    (decoder): Sequential(
      (0): resnet_block(
        (conv_1): Conv3d(128, 128, kernel_

In [5]:
v, f = igl.read_triangle_mesh(example_mesh)
v = 2*mt.NDCnormalize(v)
grid_n = 33

points = mt.mesh_grid(grid_n, True)
sdf = igl.signed_distance(points, v, f)[0].reshape(grid_n, grid_n, grid_n)

tensor_sdf = torch.tensor(sdf, dtype=torch.float32, device=device)[None, None, ...]
tensor_mask = torch.tensor(make_mask_close(sdf, grid_n), dtype=torch.bool, device=device).flatten()[None, ...]

In [6]:
ponq = CNN_to_PoNQ(model, tensor_sdf, grid_n, tensor_mask, device=device, subd=0)

In [9]:
plot(*ponq.min_cut_surface(grid_n))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0008501…

In [8]:
plot(*mt.mesh_from_voxels(sdf))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0004573…